In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import time

# Sua chave de API
api_key = 'uuuuuuuuu'
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
def get_live_chat_id(video_id):
    video_response = youtube.videos().list(
        part='liveStreamingDetails',
        id=video_id
    ).execute()

    live_chat_id = video_response['items'][0]['liveStreamingDetails'].get('activeLiveChatId')
    if not live_chat_id:
        raise ValueError("Não há live chat ativo para este vídeo.")
    return live_chat_id

def get_chat_messages(live_chat_id, page_token=None):
    messages = []
    chat_request = youtube.liveChatMessages().list(
        liveChatId=live_chat_id,
        part='snippet,authorDetails',
        maxResults=100,
        pageToken=page_token
    )
    chat_response = chat_request.execute()

    for item in chat_response['items']:
        # Adicionando verificação para garantir que a chave 'displayMessage' está presente
        message_text = item['snippet'].get('displayMessage', '---Mensagem não disponível---')
        author_name = item['authorDetails']['displayName']
        messages.append((author_name, message_text))
    
    next_page_token = chat_response.get('nextPageToken')
    return messages, next_page_token

# Substitua 'VIDEO_ID' pelo ID do vídeo da live
video_id = 'uuuuuuuuuuuuuu'
all_messages = []
next_page_token = None

try:
    live_chat_id = get_live_chat_id(video_id)
    print("Coletando mensagens do chat ao vivo...")
    while True:
        if next_page_token or not all_messages:  # Nova requisição apenas se houver mais mensagens ou se for a primeira vez
            new_messages, next_page_token = get_chat_messages(live_chat_id, next_page_token)
            all_messages.extend(new_messages)
            print(f"Novas mensagens coletadas: {len(new_messages)}")
        
        if not new_messages:
            print("Não há novas mensagens. Aguardando novas mensagens...")
        
        time.sleep(60)  # Aumentado para 20 segundos

except KeyboardInterrupt:
    print("Interrompido pelo usuário.")
finally:
    # Salvando as mensagens em um CSV antes de encerrar o script
    df = pd.DataFrame(all_messages, columns=['Author', 'Message'])
    df.to_csv('youtube_live_chat_messages.csv', index=False, encoding='utf-8')
    print(f"Mensagens do chat ao vivo salvas com sucesso no arquivo 'youtube_live_chat_messages.csv'. Total de mensagens coletadas: {len(all_messages)}")
